In [1]:
## import data manipulation packages
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
from math import radians

## import dataset as variable 'city' and drop NaN
cities = pd.read_excel('small_cities.xlsx')
ct = cities.dropna(axis = 'rows', how = 'any')

## concatenate iso2 and city to get unique id
ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)

<ipython-input-1-412402616f6d>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)


In [2]:
## drop not usable columns
ct = ct.drop(['city_ascii', 'country', 'iso2', 'admin_name', 'capital', 'id'], axis = 1)

In [3]:
#add london Start
London_st = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]
London_st['city']='London_st' 
London_st['lng'] = London_st['lng'] + 0.2



<ipython-input-3-748376aaf119>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['city']='London_st'
<ipython-input-3-748376aaf119>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['lng'] = London_st['lng'] + 0.2


In [4]:
ct.append(London_st)

,city,lat,lng,iso3,population,ID
0,Tokyo,35.6897,139.6922,JPN,37977000,TokyoJP
1,Jakarta,-6.2146,106.8451,IDN,34540000,JakartaID
2,Delhi,28.6600,77.2300,IND,29617000,DelhiIN
3,Mumbai,18.9667,72.8333,IND,23355000,MumbaiIN
4,Manila,14.5958,120.9772,PHL,23088000,ManilaPH
5,Shanghai,31.1667,121.4667,CHN,22120000,ShanghaiCN
6,São Paulo,-23.5504,-46.6339,BRA,22046000,São PauloBR
7,Seoul,37.5833,127.0000,KOR,21794000,SeoulKR
8,Mexico City,19.4333,-99.1333,MEX,20996000,Mexico CityMX
9,Guangzhou,23.1288,113.2590,CHN,20902000,GuangzhouCN


In [5]:
#population
pop = np.where(ct['population']< 200000 , 0 , 2 )
#same state 
zero_data = np.ones(shape=(len(ct),len(ct)))
st = pd.DataFrame(zero_data , columns = ct['iso3'], index = ct['iso3'])
for i in ct['iso3']:
    st.loc[ i , i ] = 0   
st = st * 2

In [6]:
#direction
dr_x = np.where(ct['lat']>= 0 , ct['lat'] , (ct['lat'] + 150) + 150 )
dr_y =np.where(ct['lat']>= 0 , ct['lat'] , (ct['lat'] + 150) + 150 )
drdf = pd.DataFrame(zero_data , columns = dr_x, index = dr_y)
for i in dr_x:
    for e in dr_y: 
        drdf.loc[i , e] = np.where( i >= e , 0 , 1 ) 

In [7]:
#metto i giusti indici alle matrici 
direction = pd.DataFrame( drdf.to_numpy() , columns = ct['ID'], index = ct['ID'])
same_state = pd.DataFrame( st.to_numpy() , columns = ct['ID'], index = ct['ID'])
popolation = pd.DataFrame( pop , index = ct['ID'])





In [8]:
## coversion of coordinated in degrees to radians
ct['lat'] = np.radians(ct['lat'])
ct['lng'] = np.radians(ct['lng'])

In [9]:
## get the scipy metrics
dist = DistanceMetric.get_metric('haversine')


In [10]:
## prepare the array to get in input for the distance calculation
ct[['lat','lng']].to_numpy()

array([[ 6.22902774e-01,  2.43808883e+00],
       [-1.08465232e-01,  1.86479878e+00],
       [ 5.00211364e-01,  1.34791778e+00],
       [ 3.31031363e-01,  1.27118089e+00],
       [ 2.54744767e-01,  2.11145046e+00],
       [ 5.43961532e-01,  2.11999385e+00],
       [-4.11032020e-01, -8.13915098e-01],
       [ 6.55952329e-01,  2.21656815e+00],
       [ 3.39175070e-01, -1.73020248e+00],
       [ 4.03673712e-01,  1.97674246e+00],
       [ 6.96473638e-01,  2.03141315e+00],
       [ 5.24577905e-01,  5.45230386e-01],
       [ 3.93416412e-01,  1.54178546e+00],
       [ 9.73122287e-01,  6.56554467e-01],
       [ 2.39982772e-01,  1.75434737e+00],
       [-6.03878685e-01, -1.01895638e+00],
       [ 3.93309947e-01,  1.99061783e+00],
       [ 4.13924031e-01,  1.57770958e+00],
       [ 8.98970228e-01, -2.22529480e-03]])

In [11]:
## pass pairwise function to get the distance / multiply for 6373 as earth sperical radius is 6373kms

dist.pairwise(ct [['lat','lng']].to_numpy())*6373

array([[    0.        ,  5787.90376514,  5835.24850156,  6736.36809942,
         2997.19433953,  1762.02238898, 18539.98524556,  1151.30788725,
        11309.38036426,  2903.80048079,  2094.45550534,  9566.23329973,
         5139.14281822,  7480.57051994,  4603.65727549, 18375.34461103,
         2875.85310361,  4891.88600698,  9561.72022915],
       [ 5787.90376514,     0.        ,  5011.26756513,  4662.28762191,
         2790.71769102,  4439.07372657, 15633.27081416,  5301.08499752,
        16850.82224408,  3337.52354223,  5222.60346539,  8988.64386526,
         3780.03420361,  9315.18347434,  2328.00380897, 15223.81716474,
         3292.7014063 ,  3777.92977517, 11722.2778931 ],
       [ 5835.24850156,  5011.26756513,     0.        ,  1166.99745941,
         4755.33798859,  4245.89964831, 14435.54333137,  4686.01564302,
        14657.70559863,  3643.36138692,  3775.41614973,  4430.87846438,
         1304.60654576,  4340.81998221,  2919.41239224, 15799.73412347,
         3741.19868618

In [36]:
## create the distance matrix with cities in the indexes
distance = pd.DataFrame(dist.pairwise(ct[['lat','lng']].to_numpy())*6373, columns = ct['ID'], index = ct['ID'])


In [37]:
## identify 'London' ID 
ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]

,city,lat,lng,iso3,population,ID
19,London,0.89897,-0.002225,GBR,10979000,LondonGB


In [38]:
## get starting line from distance matrix
distance.loc[['LondonGB']]
from pandas import DataFrame


In [39]:
#create the graph 
graph = pd.DataFrame( np.zeros(shape=(len(ct),len(ct))) , columns = ct['ID'], index = ct['ID'])

for i in distance :
    graph[i] = distance[i]*direction[i]

In [40]:
## popolare per criteri (più vicino +2, secondo più vicino +4, terzo più vicino +8, il resto 'no go')
dis = graph.replace(0, 0)
dis = dis.replace(dis.apply(lambda x: x[x > 0].min(axis=0)), 2)
dis = dis.replace(dis.apply(lambda x: x[x > 2].min(axis=0)), 4)
dis = dis.replace(dis.apply(lambda x: x[x > 4].min(axis=0)), 8)
#dis = dis.replace(dis.apply(lambda x: x[x > 8]), 'no go')

In [41]:
#parte messa solo perchè l'utima riga sopra non mi va 
dis = dis.where( (dis <= 8)  , 0  ) 
dis

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,BeijingCN,CairoEG,KolkātaIN,MoscowRU,BangkokTH,Buenos AiresAR,ShenzhenCN,DhakaBD,LondonGB
ID,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,8.0,0.0,0.0,0.0,0.0,4.0
MumbaiIN,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,8.0,8.0,8.0
ManilaPH,0.0,4.0,0.0,4.0,0.0,8.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
ShanghaiCN,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
São PauloBR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SeoulKR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Mexico CityMX,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [42]:
for i in dis :
    graph[i] = (dis[i] + same_state[i] + pop) * dis[i] / dis[i]
    
graph

#gr.T.tolist

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,BeijingCN,CairoEG,KolkātaIN,MoscowRU,BangkokTH,Buenos AiresAR,ShenzhenCN,DhakaBD,LondonGB
ID,,,,,,,,,,,,,,,,,,,
TokyoJP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
JakartaID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DelhiIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,12.0,NaN,NaN,NaN,NaN,8.0
MumbaiIN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,6.0,6.0,NaN,NaN,NaN,12.0,12.0,12.0
ManilaPH,NaN,8.0,NaN,8.0,NaN,12.0,NaN,NaN,6.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN
ShanghaiCN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
São PauloBR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SeoulKR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mexico CityMX,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN


In [48]:
#parte messa solo perchè l'utima riga sopra non mi va 

graph = graph.where( (graph > 1 )  , 0  ) 

gr=np.array(graph).T

In [62]:
gr

array([[ 0.,  0.,  0.,  0.,  0.,  6.,  0.,  0.,  0., 12.,  0.,  0.,  0.,
         0.,  0.,  0.,  8.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  6.,  0., 12.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  6.,
         0.,  0.,  0.,  0., 12.,  0.],
       [ 0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  6.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  6.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 12.,  0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,
         0.,  0.,  0.,  6.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  8.,  0.,  0.,  0.,  0.,
         0.,  0.,  6.,  0.,  0., 12.],
       [ 8.,  0.,  0.,  0.,  0.,  6.,  0.,  0.,  0., 12.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  8.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0., 12.,  0.,  0.,  0.

In [45]:
import sys 

In [60]:
# Library for INT_MAX 
import sys 
   
class Graph(): 
   
    def __init__(self, vertices): 
        self.V = vertices 
        self.graph = [[0 for column in range(vertices)]  
                    for row in range(vertices)] 
   
    def printSolution(self, dista): 
        print ("Vertex tDistance from Source") 
        for node in range(self.V): 
            print (node, "t", dista[node]) 
   
    # A utility function to find the vertex with  
    # minimum distance value, from the set of vertices  
    # not yet included in shortest path tree 
    def minDistance(self, dista, sptSet): 
   
        # Initilaize minimum distance for next node 
        min = sys.maxsize 
   
        # Search not nearest vertex not in the  
        # shortest path tree 
        for v in range(self.V): 
            if dista[v] < min and sptSet[v] == False: 
                min = dista[v] 
                min_index = v 
   
        return min_index 
   
    # Funtion that implements Dijkstra's single source  
    # shortest path algorithm for a graph represented  
    # using adjacency matrix representation 
    def dijkstra(self, src): 
   
        dista = [sys.maxsize] * self.V 
        dista[src] = 0
        sptSet = [False] * self.V 
   
        for cout in range(self.V): 
   
            # Pick the minimum distance vertex from  
            # the set of vertices not yet processed.  
            # u is always equal to src in first iteration 
            u = self.minDistance(dista, sptSet) 
   
            # Put the minimum distance vertex in the  
            # shotest path tree 
            sptSet[u] = True
   
            # Update dist value of the adjacent vertices  
            # of the picked vertex only if the current  
            # distance is greater than new distance and 
            # the vertex in not in the shotest path tree 
            for v in range(self.V): 
                if self.graph[u][v] > 0 and \
                     sptSet[v] == False and \
                     dista[v] > dista[u] + self.graph[u][v]: 
                     dista[v] = dista[u] + self.graph[u][v] 
   
        self.printSolution(dista) 
   
# Driver program 

  
# This code is contributed by Divyanshu Mehta 

In [61]:
G = Graph(5) 
G.graph = gr
   
G.dijkstra(0); 

UnboundLocalError: local variable 'min_index' referenced before assignment

In [23]:
import sys

# Function to find out which of the unvisited node 
# needs to be visited next
def to_be_visited():
  global visited_and_distance
  v = -10
  # Choosing the vertex with the minimum distance
  for index in range(number_of_vertices):
    if visited_and_distance[index][0] == 0 \
      and (v < 0 or visited_and_distance[index][1] <= \
      visited_and_distance[v][1]):
        v = index
  return v

# Creating the graph as an adjacency matrix
vertices = [[0, 1, 1, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [0, 0, 0, 0]]
edges =  [[0, 3, 4, 0],
          [0, 0, 0.5, 0],
          [0, 0, 0, 1],
          [0, 0, 0, 0]]

number_of_vertices = len(vertices[0])
